# Algebra of Programming
- Dijkstra
- backhouse
- http://www.mathmeth.com/read.shtml
- Bird book
- AoP book
- Gries https://www.cs.cornell.edu/fbs/publications/EqPropLogic.pdf


https://arxiv.org/abs/2312.06103 equatonal reasoning dependent types. references gibbons paper
https://leahneukirchen.org/caudex/equational-reasoning.html great links

Triggers and evaluation order. There is a sense that we want rules to be simplifying. In coq etc one cares alot about computational content. Z3 triggers are a way to approach this. 

In [ ]:
!pip install git+https://github.com/philzook58/knuckledragger.git


<a target="_blank" href="https://colab.research.google.com/github/philzook58/knuckledragger.git">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Open in google colab <https://colab.research.google.com/github/philzook58/knuckledragger/blob/main/theories/equational.ipynb>

There are examples of `calc` tactics elsewhere in lean and dafny. Rather than using the raw inference rules, we can write down an equational proof 

In [25]:
from knuckledragger.kernel import infer
class Calc():
    def __init__(self, lhs, rhs, ctx = []):
        self.terms = [lhs]
        self.thm = infer(ctx, lhs == lhs)
        self.thms = []
        self.goal = rhs
        self.ctx = ctx
    def then(self, rhs, by=[]): # step? rw?
        thm = infer(by + self.ctx, self.terms[-1] == rhs)
        self.thm = infer([thm, self.thm], self.terms[0] == rhs)
        self.thms.append(thm)
        self.terms.append(rhs)
        return self
    def __eq__(self,rhs): #sadly the way python expands multiple == this does not work that great. needs parens
        return self.then(rhs)
    def simp(): # use z3 simplify. Weak though becaus can't be context aware
        self.
        return self
    def egg_simp(self):
        return self
    def __repr__(self):
        return " == ".join(map(repr, self.terms))
    def __str__(self):
        return repr(self)
    def sympy():
        return self
        pass # use sympy to simplify, use z3 to confirm
    def qed(self, by=[]):
        if self.terms[-1] is not self.goal:
            self.then(self.goal, by=by)
        return self.thm
    
def calc(*steps): # a different syntax for calc
    pf = Calc(steps[0], steps[-1])
    for t in steps[1:-1]:
        pf = pf.then(t)
    return pf

In [29]:
from knuckledragger.kernel import trust
from z3 import *
Atom = DeclareSort("Atom")
a,b,c,d = Consts("a b c d", Atom)
ab = trust(a == b)
bc = trust(b == c)
cd = trust(c == d)
pf = Calc(a, d).then(b, by=[ab]).then(c, by=[bc]).then(d, by=[cd])
ad = pf.qed()
ad
#Calc(a,d, ctx=[ab,bc,cd]) == b == c == d 
((Calc(a,d, ctx=[ab,bc,cd]) == b) == c) == d

a == b == c == d

You can also calc chain equalities of boolean expressions that are the same.
Calc also supports an inequality modes


Adding forall.

# Equational Propositional Logic
https://www.cs.cornell.edu/fbs/publications/EqPropLogic.pdf
table 2





In [ ]:
Prop = DeclareSort("Prop")
eq = Function(Prop, Prop, Prop)
true = Const("true", Prop)
false = Const("false", Prop)
_not = Function(Prop, Prop)
_and = Function(Prop, Prop, Prop)
_or = Function(Prop, Prop, Prop)
_implies = Function(Prop, Prop, Prop)

#eq_reflect =  trust(Implies(eq(a,b), a == b))
eq_reflect = trust(eq(a,b) == (a == b))
eq_assoc = trust(eq(eq(a,b),c) == eq(a,eq(b,c)))
eq_sym = trust(eq(a,b) == eq(b,a))
eq_id = trust(eq(true,eq(a,a)))
false_def = trust(eq(false, _not(true)))
not_eq_distr = trust(_not(eq(a,b)) == eq(_not(a), b))

axioms = [eq_assoc, eq_sym, eq_id, false_def, not_eq_distr]

def not_eq(a,b): # macro definition instead of first class.
    return _not(eq(a,b))

Calc(eq(_and(p, _or(p,q)) p))




# Concrete Mathematics

Josephus
Recurrences
Using sympy solutions
hypergeometric

In [ ]:

_sum = Function("sum", ArraySort(IntSort(), RealSort()), IntSort(), IntSort(), RealSort())
def Sum(c_i, i, a, b):
    return _sum(Lambda([i], c_i), a, b)
sum_base = trust(ForAll([c_i, a], (c_i, a, a) == 0))
sum_succ = trust(ForAll([c_i, a ,b], _sum(c_i, a, b+1) == _sum(c_i, a, b) + c_i[b]))

#induct = trust()

# sum_all = Function("sumall", ArraySort(IntSort(), RealSort()), RealSort()) infinite sum. Harder to treat. Non computational in some sense.
# sum_set Function("sumset", ArraySort(IntSort(), RealSort()), ListSort(IntSort()), RealSort())) # finite list
# possibly ifninite set?
#sum_set = Function("sumset", ArraySort(IntSort(), RealSort()), ArraySort(IntSort(), BoolSort()), ArraySort(IntSort(), RealSort()))



In [34]:
# we need to axiomatize pow because Z3 power cannot handle variables in the exponent
x = Real("x")
n = Int("n")
pow = Function("pow", RealSort(), IntSort(), RealSort())
pow_def = trust(ForAll([x,n], pow(x,n) == If(n == 0, 1, 
                                          If(n < 0 , pow(x,n+1) / x, 
                                                     x*pow(x,n-1)))))
# can't do this because recursive definition. Fine.
#pow, pow_def = define("pow", Lambda([x,n], If(n == 0, 1, 
#                               If(n < 0 , pow(x,n+1) / x, 
#                                          x*pow(x,n-1)))))

pow_zero = infer([pow_def], ForAll([x], pow(x,0) == 1))
pow_one = infer([pow_def, pow_zero], ForAll([x], pow(x,1) == x))
pow_one = infer([pow_def, pow_one], ForAll([x], pow(x,1) == x))

# axiom specialized to particular powers
pown = [pow_zero]
for i in range(1,5):
    pown.append(infer([pow_def, pown[-1]], ForAll([x], pow(x,i) == x**i)))
pown

[(7416851326278927786, ForAll(x, pow(x, 0) == 1)),
 (3910822737024964705, ForAll(x, pow(x, 1) == x**1)),
 (-8021081544531509068, ForAll(x, pow(x, 2) == x**2)),
 (9063170249293680640, ForAll(x, pow(x, 3) == x**3)),
 (8460666806402329999, ForAll(x, pow(x, 4) == x**4))]

In [ ]:
fall_pow